# Conversión de los modelos a Tflite.

Este cuaderno tiene la finalidad de convertir el modelo de detección y de clasifiación a TFlite, de manera que se pueda utilizar en una aplicación móvil desarrollada con Flutter.

El **detector** tiene la extensión `.pt`, por lo que primero se debe convertir a `.onnx` para poder cambiarse después a `.tf` y, por último, conseguir el `.tflite`.

El **clasificador** tiene la extensión `.keras`, por lo que se puede pasar directamente.

## Instalaciones/Imports

In [ ]:
!pip install onnx2tf
!pip install onnx
!pip install onnx-graphsurgeon
!pip install sng4onnx
!pip install ultralytics
!pip install onnxslim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.2/446.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.9/921.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
import tensorflow as tf
from tensorflow.keras.models import load_model

## .pt a .onnx

Para realizar este paso, es necesario contar con el modelo `.pt`, en nuestro caso, *best.pt*.

In [ ]:
!yolo export model=best.pt format=onnx opset=12

Ultralytics 8.3.80 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

ONNX: starting export with onnx 1.17.0 opset 12...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success ✅ 2.1s, saved as 'best.onnx' (11.7 MB)

Export complete (2.9s)
Results saved to /content
Predict:         yolo predict task=detect model=best.onnx imgsz=640  
Validate:        yolo val task=detect model=best.onnx imgsz=640 data=modificado_copy/data.yaml  
Visualize:       https://netron.app
💡 Learn more at https://docs.ultralytics.com/modes/export


## .onnx a .tf

Una vez conseguido el `.onnx` hay que pasarlo a `.tf`.

A diferencia del anterior, que nos daba el *best.onnx* directamente descargado, en este paso se descargará una carpeta con el nombre que hayamos puesto (*detector_tf*).

In [ ]:
!onnx2tf -i best.onnx -o detector_tf

E0000 00:00:1740587746.391628    4329 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740587746.406094    4329 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

Model optimizing started ============================================================
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/onnx2tf/onnx2tf.py", line 652, in convert
    result = subprocess.check_output(
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 466, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fi

## .tf a .tflite

Por último, pasamos el `.tf` a `.tflite` introduciendo el nombre del directorio creado.

In [ ]:
# Carga el modelo TensorFlow
converter = tf.lite.TFLiteConverter.from_saved_model("detector_tf")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]

# Convierte a TFLite
tflite_model = converter.convert()

# Guarda el modelo convertido
with open("detector.tflite", "wb") as f:
    f.write(tflite_model)

print("Conversión completada: detector.tflite")

Conversión completada: detector.tflite


## .keras a .tflite

Para convertir el modelo de clasificación de `.keras` a `.tflite` solo necesitamos ejecutar la siguiente celda, que nos descargará el modelo directamente.

In [13]:
# Carga el modelo .keras
model = load_model("clasificador.keras")

# Configura la conversión para usar operadores compatibles
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()

# Guarda el modelo convertido
with open("clasificador.tflite", "wb") as f:
    f.write(tflite_model)

print("Conversión completada: clasificador.tflite")


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Saved artifact at '/tmp/tmp77jnuktu'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_3')
Output Type:
  TensorSpec(shape=(None, 100), dtype=tf.float32, name=None)
Captures:
  132226848918608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132226848919376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132226848919952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132226848921296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132226848921104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132226848917456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132226848921872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132226848922640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132226848923024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132226848924368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132226848922256: 

In [14]:
interpreter = tf.lite.Interpreter(model_path="clasificador.tflite")
interpreter.allocate_tensors()
print("Modelo TFLite cargado correctamente.")


Modelo TFLite cargado correctamente.
